In [1]:
import re

# 1️⃣ إزالة التشكيل
def remove_diacritics(text):
    arabic_diacritics = re.compile(r"""
        ّ | َ | ً | ُ | ٌ | ِ | ٍ | ْ
    """, re.VERBOSE)
    return re.sub(arabic_diacritics, '', text)


# 2️⃣ توحيد الحروف
def normalize_arabic(word):
    word = re.sub("[إأآا]", "ا", word)
    word = re.sub("ى", "ي", word)
    word = re.sub("ؤ", "و", word)
    word = re.sub("ئ", "ي", word)
    word = re.sub("ة", "ه", word)
    return word


# 3️⃣ السوابق واللواحق
prefixes = ["وال", "بال", "كال", "فال", "لل", "ال", "و", "ف", "ب", "ك", "ل"]
suffixes = ["ات", "ون", "ين", "ان", "ه", "ها", "هم", "نا", "ي"]


def remove_prefix(word):
    for p in prefixes:
        if word.startswith(p) and len(word) > len(p) + 2:
            return word[len(p):]
    return word


def remove_suffix(word):
    for s in suffixes:
        if word.endswith(s) and len(word) > len(s) + 2:
            return word[:-len(s)]
    return word


# 4️⃣ الأوزان الصرفية (Khoja Patterns)
patterns = {
    "فاعل": [0, 2, 4],
    "مفعل": [1, 3, 4],
    "فعول": [0, 2, 3],
    "فعيل": [0, 2, 3],
    "مفعول": [1, 3, 4],
    "تفعيل": [1, 3, 4],
    "افتعال": [1, 3, 5],
    "استفعال": [2, 4, 6]
}


# 5️⃣ استخراج الجذر
def extract_root(word):
    for pattern, indexes in patterns.items():
        if len(word) == len(pattern):
            try:
                root = ''.join(word[i] for i in indexes)
                return root
            except:
                pass
    return word


# 6️⃣ Khoja Stemmer النهائي
def khoja_stemmer(word):
    word = remove_diacritics(word)
    word = normalize_arabic(word)
    word = remove_prefix(word)
    word = remove_suffix(word)
    return extract_root(word)


# 🧪 اختبار
if __name__ == "__main__":
    words = [
        "المكتبات",
        "والمدرسون",
        "الاستغفار",
        "المهندسين",
        "الطالبات"
    ]

    for w in words:
        print(w, "→", khoja_stemmer(w))


المكتبات → متب
والمدرسون → مرس
الاستغفار → تفر
المهندسين → هدس
الطالبات → طلب


In [2]:
import re

# ===============================
# 1️⃣ إزالة التشكيل
# ===============================
def remove_diacritics(text):
    return re.sub(r"[ًٌٍَُِّْـ]", "", text)


# ===============================
# 2️⃣ توحيد الحروف
# ===============================
def normalize_arabic(word):
    substitutions = {
        "أ": "ا", "إ": "ا", "آ": "ا",
        "ى": "ي",
        "ؤ": "و",
        "ئ": "ي",
        "ة": "ه"
    }
    for k, v in substitutions.items():
        word = word.replace(k, v)
    return word


# ===============================
# 3️⃣ السوابق واللواحق
# ===============================
prefixes = ["وال", "بال", "كال", "فال", "لل", "ال", "و", "ف", "ب", "ك", "ل", "س"]
suffixes = ["ات", "ون", "ين", "ان", "ها", "هم", "نا", "ي", "ه"]

internal_letters = ["ا", "ت", "م", "س", "ن", "ي"]


def remove_prefixes(word):
    for p in prefixes:
        if word.startswith(p) and len(word) - len(p) >= 3:
            word = word[len(p):]
    return word


def remove_suffixes(word):
    for s in suffixes:
        if word.endswith(s) and len(word) - len(s) >= 3:
            word = word[:-len(s)]
    return word


# ===============================
# 4️⃣ إزالة الزوائد الداخلية
# ===============================
def remove_internal_letters(word):
    if len(word) > 3:
        for l in internal_letters:
            if l in word[1:-1]:
                temp = word.replace(l, "", 1)
                if len(temp) >= 3:
                    return temp
    return word


# ===============================
# 5️⃣ التحقق من الجذر
# ===============================
def is_valid_root(root):
    return len(root) == 3 and root.isalpha()


# ===============================
# 6️⃣ Khoja Stemmer المحسّن
# ===============================
def khoja_stemmer_improved(word):
    word = remove_diacritics(word)
    word = normalize_arabic(word)

    word = remove_prefixes(word)
    word = remove_suffixes(word)

    # محاولة بدون إزالة داخلية
    if is_valid_root(word):
        return word

    # محاولة مع إزالة داخلية
    word2 = remove_internal_letters(word)
    if is_valid_root(word2):
        return word2

    # آخر حل: أول 3 أحرف
    return word[:3]


# ===============================
# 🧪 اختبار
# ===============================
if __name__ == "__main__":
    words = [
        "المكتبات",
        "والمدرسون",
        "الاستغفار",
        "المهندسين",
        "المستخرج",
        "الطالبات"
    ]

    for w in words:
        print(w, "→", khoja_stemmer_improved(w))


المكتبات → مكب
والمدرسون → مدر
الاستغفار → است
المهندسين → مهن
المستخرج → مست
الطالبات → طلب
